In [41]:
import numpy as np
import pandas as pd

from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker


# LOAD DATA (csv have been adapted to be separated by ';' since some names include ',' already)
# (node, type)
data_nodes = pd.read_csv("data/MarvelUniverse/nodes.csv", sep=";").reset_index()

# (hero, comic)
data_comic_edges = pd.read_csv("data/MarvelUniverse/edges.csv", sep=";")

# (hero1, hero2)
data_hero_edges = pd.read_csv("data/MarvelUniverse/hero-network.csv", sep=";")

# == CREATE FIRST KNOWLEDGE GRAPH ==================================================================
knowledge_graph = KG()

# create vertices / nodes
for index, row in data_nodes.iterrows():
    vertex = Vertex(row["node"])
    check = knowledge_graph.add_vertex(vertex)
    if not check:
        print(row["node"])

# create edges / walks
# create hero - comic edges / walks
walks = []
for index, row in data_comic_edges.iterrows():
    hero = Vertex(row["hero"])
    comic = Vertex(row["comic"])
    pred = Vertex("appears in")
    check = knowledge_graph.add_walk(hero, pred, comic)
    if not check:
        print(row["hero"], row["comic"])
    else:
        walks.append([hero, comic, pred])

# create a transformer
transformer = RDF2VecTransformer()
transformer.fit(walks)
walk_embeddings = transformer.transform(knowledge_graph, knowledge_graph._entities)
print(len(knowledge_graph._entities), np.array(walk_embeddings).shape)

# URL = "http://pyRDF2Vec"
# CUSTOM_KG = KG()

# for row in GRAPH:
#     subj = Vertex(f"{URL}#{row[0]}")
#     obj = Vertex((f"{URL}#{row[2]}"))
#     pred = Vertex((f"{URL}#{row[1]}"), predicate=True, vprev=subj, vnext=obj)
#     CUSTOM_KG.add_walk(subj, pred, obj)


TypeError: 'Vertex' object is not iterable